[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/H-IAAC/d3vis_ipynb/blob/main/examples/graphs.ipynb)

# Import Datasets

In [ ]:
from sklearn.svm import OneClassSVM
import pandas as pd
from sklearn.ensemble import IsolationForest
import numpy as np
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
import shap
import gdown
import os
from d3vis_ipynb import BarPlot, HistogramPlot, ScatterPlot, LinearPlot, RangeSlider, MatrixLayout, MapPlot, BeeswarmPlot

In [ ]:
output_path = "data/dados_saeb.csv"
 
if not os.path.isfile(output_path):
    if not os.path.exists("data"):
        os.makedirs("data")
    file_id = "1ClwCEiXc2bqM5bZ__u1Zxw0Q39yJDqjE"
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
 
df = pd.read_csv(output_path, encoding='utf-8', sep=';')

In [ ]:
df.columns = df.columns.str.replace("ALUNOS_PORCENTAGEM_", "", regex=False)
df.columns = df.columns.str.replace("PROFESSOR_PORCENTAGEM_", "", regex=False)

In [ ]:
df_drop = df.drop(['MEDIA_EM_MT', 'MEDIA_EM_LP', 'ALUNOS_ID_ESCOLA'], axis=1)
X = df_drop
y = df['MEDIA_EM_MT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

parameters = {
    'lambda': 0.007342571956423321, 
    'alpha': 0.0049894947342883815, 
    'colsample_bytree': 0.5, 
    'subsample': 0.7, 
    'learning_rate': 0.03263662172830031, 
    'n_estimators': 801, 
    'max_depth': 4, 
    'min_child_weight': 9, 
    'gamma': 0.0014832545774497129
}


model = XGBRegressor(**parameters)#LGBMRegressor(**parameters)
# fit model
model.fit(X_train, y_train)

In [ ]:
# compute SHAP values
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

In [ ]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': np.abs(shap_values.values).mean(axis=0)
})

# Ordenando e selecionando as 20 mais importantes
top_20_features = feature_importance.sort_values(by='importance', ascending=False).head(20)
important_features = top_20_features['feature'].tolist()

# Identificando os índices das 20 principais features no array
important_indices = [list(X.columns).index(feature) for feature in important_features]

# Filtrando shap_values para as 20 principais características
filtered_shap_values = shap_values[:, important_indices]

# BeeswarmPlot

Generates a beeswarm plot from data.

Parameters:
- **explanation**: a shap's Explanation object

In [ ]:
from d3vis_ipynb import BeeswarmPlot
beeswarmPlot = BeeswarmPlot(explanation=filtered_shap_values[:500])
beeswarmPlot

# DecisionPlot

Generates a decision plot from data.

Parameters:
- **explanation**: a shap's Explanation object

In [ ]:
from d3vis_ipynb import DecisionPlot
decisionPlot = DecisionPlot(explanation=filtered_shap_values[:50])
decisionPlot

# ForcePlot

Generates a waterfall plot from data.

Parameters:
- **baseValue**: a float with the base value
- **explanation**: a shap's Explanation object

Can be initialized two different ways: with a *shap._explanation.Explanation* object or with a *pandas.core.frame.DataFrame* object.

In [ ]:
from d3vis_ipynb import ForcePlot
forcePlot = ForcePlot(explanation=filtered_shap_values[0])
forcePlot

Click on the polygons and run the cell bellow:

# WaterfallPlot

Generates a waterfall plot from data.

Parameters:
- **baseValue**: a float with the base value
- **explanation**: a shap's Explanation object

Can be initialized two different ways: with a *shap._explanation.Explanation* object or with a *pandas.core.frame.DataFrame* object.

In [ ]:
from d3vis_ipynb import WaterfallPlot
waterfallPlot = WaterfallPlot(explanation=filtered_shap_values[0])
waterfallPlot

Click on the polygons and run the cell bellow:

In [ ]:
# waterfallPlot.selectedValues